In [2]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras import losses
import tensorflowjs as tfjs
import os
import pickle
from keras_tuner.tuners import RandomSearch
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
print(tf.config.list_physical_devices('GPU'))

[]


In [ ]:
# Constants
BATCH_SIZE = 32
LR = 1e-4

In [ ]:
# Load training data
with open('data/matches/diamond.ds', 'rb') as f:
    training_data = pickle.load(f)

In [ ]:
# Partition Dataset
train_len = int(training_data[0].shape[0] * 0.9)

train_x = training_data[0][:train_len]
train_y = training_data[1][:train_len]

test_x = training_data[0][train_len:]
test_y = training_data[1][train_len:]

In [ ]:
def model_builder(hp):
    model = keras.Sequential()
    for i in range(hp.Int('layers', 4, 12)):
        model.add(layers.Dense(
            units=hp.Int('units_'+str(i), 50, 1000, step=100),
            activation=hp.Choice('act_'+str(i), ['relu', 'elu', 'selu'])))
    model.add(tf.keras.layers.Dense(1))
    
    loss = losses.MeanAbsoluteError()
    optimizer = optimizers.Adam(learning_rate=hp.Choice('learning_rate', values=[1e-2, 1e-4]))
    model.compile(loss=loss, optimizer=optimizer)
    return model

In [ ]:
tuner = RandomSearch(
    model_builder,
    objective='val_loss',
    max_trials=100,
    executions_per_trial=1,
    directory='checkpoints',
    project_name='winpredictor'
)

In [ ]:
tuner.search_space_summary()

In [ ]:
tuner.search(train_x[:10000], train_y[:10000], epochs=5, validation_data=(test_x[:1000], test_y[:1000]))

In [ ]:
history = model.fit(train_x, train_y, batch_size=BATCH_SIZE, validation_split=0.01, epochs=200, verbose=1)

In [ ]:
history_dict = history.history

In [ ]:
plt.figure(figsize=(5, 5))
plt.plot(history_dict['loss'], label='Training Loss')
plt.plot(history_dict['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [36]:
tfjs.converters.save_keras_model(export_model, 'winpredictor/public/models/master')

In [4]:
ranks = ['grandmaster', 'master', 'diamond', 'platinum', 'gold', 'silver', 'bronze', 'iron']

for r in ranks:
    model = tf.keras.models.load_model(f'models/{r}.h5', compile=False)
    tfjs.converters.save_keras_model(model, f'winpredictor/public/models/{r}')